In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.7 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
path_base = '/content/drive/MyDrive/Tesis/SICK/transformer_tunned_BERT/uncase_base/study_optuna/'
file_root = '/content/drive/MyDrive/Tesis/SICK/uncase_based/SICK.txt'
with open(file_root, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split("\t")) for l in temp_f.readlines() ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getSICKSents(filename='sts-train.csv', set_type='TRAIN', encoding='utf-8'):
  f = open(filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for i, line in enumerate(f):
    example = re.split(r'\t+', line)
    if i >= 1:
      if example[-1].strip() == set_type:
        s2.append(example[1])
        s1.append(example[2])
        target.append(float(example[4]))
  print("samples: {}".format(len(target)))
  return s1, s2, target

In [ ]:
s1_train,s2_train,target_train = getSICKSents(file_root, 'TRAIN')

samples: 4439


In [ ]:
s1_test,s2_test,target_test= getSICKSents(file_root, 'TEST')

samples: 4906


In [ ]:
s1_dev,s2_dev,target_dev= getSICKSents(file_root, 'TRIAL')

samples: 495


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [ ]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'SICK'
BERT_PATH = "bert-base-uncased"

In [ ]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [ ]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [ ]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [ ]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order.
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [ ]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 139
Number of evaluation batches: 16


In [ ]:
class BertBaseUncasedRegressor(nn.Module):
  def __init__(self, num_classes = 1, dropout=0.1, attentions=True, n_layers = 0):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      # Return:
      # last_hidden_state.shape -> [batch_size, num_tokens_in_sequence,hidden_size] (bert_base hidden_size = 768)
      # pooler_output.shape -> [batch_size, hidden_size] se utiliza una capa de pooling simple que aplica una transformación
      # lineal seguida de una función de activación tangente hiperbólica (tanh) a la última representación oculta (last_hidden_state) del token [CLS].
      layers = []
      for _ in range(n_layers):
          layers.append(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)) #self.bert.config.hidden_size -> nos da el tamaño oculto
          layers.append(nn.Dropout(dropout))
      layers.append(nn.Linear(self.bert.config.hidden_size, num_classes)) #self.bert.config.hidden_size -> nos da el tamaño oculto

      self.regressor = nn.Sequential(*layers)


  def forward(self, input_ids, type_ids, mask):
      bert_output =self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      attention_mask = mask.unsqueeze(-1).expand(bert_output.last_hidden_state.shape).float()
      embeddings = bert_output.last_hidden_state * attention_mask
      embeddings_sum = torch.sum(embeddings, dim=1)
      counts = torch.clamp(attention_mask.sum(1), min=1e-9)
      output_mean_pooling = embeddings_sum / counts
      #output = bert_output.pooler_output
      #output = bert_output.last_hidden_state[:, 0, :]
      logits = self.regressor(output_mean_pooling)
      #logits = self.regressor(output_drop)
      return logits, bert_output

In [ ]:
pp = torch.tensor([[[1,2,3],[4,5,6]],[[11,12,13],[14,15,16]]])
pp2 = torch.tensor([[[1,1,1],[1,1,0]],[[1,1,1],[1,0,0]]])
ma = pp*pp2
sum = torch.sum(ma, dim=1)
print(ma)
print(ma.shape)
print(sum)
print(sum.shape)
counts = torch.clamp(pp2.sum(1), min=1e-9)
print(counts)
print(sum/counts)

In [ ]:
"""model = BertBaseUncasedRegressor().to(device)"""

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
"""
criterion = nn.MSELoss()
LEARNING_RATE = 0.0001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
"""

In [ ]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [ ]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [ ]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)



    for i, batch in enumerate(loader):

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman


In [ ]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)


    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())

        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)

    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [ ]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")

      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()

      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [ ]:
study_name = 'study-bert-mean-pooling-base-sick'

In [ ]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [ ]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 0, 2),
      "dropout": trial.suggest_uniform("dropout", 0, 0.7),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedRegressor(dropout=params["dropout"], n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

  NUM_EPOCHS = 5
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)



  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):

    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [ ]:
storage_path = path_base + "optuna_study_mean_pooling_SICK_.db"

In [ ]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=f"sqlite:///{storage_path}", load_if_exists=True)

[I 2023-11-17 19:32:57,348] Using an existing study with name 'study-bert-mean-pooling-base-sick' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=15)

<ipython-input-27-f4670a8167f8>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-27-f4670a8167f8>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 93.6108s | Train loss: 0.6434 | Dev loss: 0.3401
Train Pearson Coef: 0.6586 | Dev Pearson Coef: 0.8636
Train Spearman Coef: 0.6147 | Dev Spearman Coef: 0.8158
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 94.9673s | Train loss: 0.1975 | Dev loss: 0.2554
Train Pearson Coef: 0.9014 | Dev Pearson Coef: 0.8800
Train Spearman Coef: 0.8479 | Dev Spearman Coef: 0.8280
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 94.5764s | Train loss: 0.1117 | Dev loss: 0.3860
Train Pearson Coef: 0.9456 | Dev Pearson Coef: 0.8730
Train Spearman Coef: 0.9155 | Dev Spearman Coef: 0.8186
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 94.7673s | Train loss: 0.0689 | Dev loss: 0.2523
Train Pearson Coef: 0.9670 | Dev Pearson Coef: 0.8744
Train Spearm

[I 2023-11-17 19:41:18,359] Trial 15 finished with value: 0.2523246016353369 and parameters: {'num_layer': 0, 'dropout': 0.3756659966495096, 'optimizer': 'RMSprop', 'learning_rate': 2.0987793186969648e-05}. Best is trial 14 with value: 0.22891508135944605.
<ipython-input-27-f4670a8167f8>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-27-f4670a8167f8>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 94.6446s | Train loss: 0.5899 | Dev loss: 0.2465
Train Pearson Coef: 0.6891 | Dev Pearson Coef: 0.8744
Train Spearman Coef: 0.6432 | Dev Spearman Coef: 0.8202
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 94.7717s | Train loss: 0.1837 | Dev loss: 0.3916
Train Pearson Coef: 0.9089 | Dev Pearson Coef: 0.8714
Train Spearman Coef: 0.8605 | Dev Spearman Coef: 0.8141
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 94.5913s | Train loss: 0.0964 | Dev loss: 0.3673
Train Pearson Coef: 0.9534 | Dev Pearson Coef: 0.8801
Train Spearman Coef: 0.9291 | Dev Spearman Coef: 0.8311
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 94.7188s | Train loss: 0.0609 | Dev loss: 0.2857
Train Pearson Coef: 0.9708 | Dev Pearson Coef: 0.8789
Train Spearm

...

--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 94.5379s | Train loss: 0.0680 | Dev loss: 0.2289
Train Pearson Coef: 0.9676 | Dev Pearson Coef: 0.8808
Train Spearman Coef: 0.9521 | Dev Spearman Coef: 0.8298
--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 95.0212s | Train loss: 0.7988 | Dev loss: 0.2683
Train Pearson Coef: 0.5748 | Dev Pearson Coef: 0.8630
Train Spearman Coef: 0.5167 | Dev Spearman Coef: 0.8206
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 94.8607s | Train loss: 0.2319 | Dev loss: 0.3543
Train Pearson Coef: 0.8840 | Dev Pearson Coef: 0.8741
Train Spearman Coef: 0.8244 | Dev Spearman Coef: 0.8274
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 94.9492s | Train loss: 0.1396 | Dev loss: 0.4565
Train Pearson Coef: 0.9321 | Dev Pearson Coef: 0.8750
Train Spearm

[I 2023-11-17 21:24:57,158] Trial 28 finished with value: 0.2683083498850465 and parameters: {'num_layer': 1, 'dropout': 0.4790996483628157, 'optimizer': 'RMSprop', 'learning_rate': 2.7597108923430253e-05}. Best is trial 22 with value: 0.21849243296310306.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 94.6782s | Train loss: 0.0735 | Dev loss: 0.2704
Train Pearson Coef: 0.9650 | Dev Pearson Coef: 0.8806
Train Spearman Coef: 0.9484 | Dev Spearman Coef: 0.8314
--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 92.2411s | Train loss: 12.7658 | Dev loss: 12.7503
Train Pearson Coef: 0.0073 | Dev Pearson Coef: 0.0628
Train Spearman Coef: -0.0006 | Dev Spearman Coef: 0.0456
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 92.5292s | Train loss: 12.5313 | Dev loss: 12.5065
Train Pearson Coef: 0.0086 | Dev Pearson Coef: 0.0654
Train Spearman Coef: 0.0012 | Dev Spearman Coef: 0.0514
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 92.3334s | Train loss: 12.2981 | Dev loss: 12.2640
Train Pearson Coef: 0.0098 | Dev Pearson Coef: 0.0675
Train

[I 2023-11-17 21:32:53,666] Trial 29 finished with value: 11.782268702983856 and parameters: {'num_layer': 1, 'dropout': 0.37585747726213736, 'optimizer': 'SGD', 'learning_rate': 3.238480346711476e-05}. Best is trial 22 with value: 0.21849243296310306.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 92.3318s | Train loss: 11.8349 | Dev loss: 11.7823
Train Pearson Coef: 0.0121 | Dev Pearson Coef: 0.0714
Train Spearman Coef: 0.0066 | Dev Spearman Coef: 0.0666


In [ ]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics).dropna(subset=['train_loss'])
print(len(df_optuna))
print(len(df_metrics ))

30
30


In [ ]:
#df_metrics = df_metrics[['train_loss', 'valid_loss', 'pearson_train', 'pearson_val', 'spearman_train', 'spearman_val']]

In [ ]:
#df_metrics = df_metrics.rename(columns={'train_loss': 'train_loss', 'valid_loss': 'val_loss', 'pearson_train': 'Pearson_train','pearson_val': 'Pearson_val', 'spearman_train': 'Spearman_train', 'spearman_val': 'Spearman_val'})

In [ ]:
df_metrics

In [ ]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

Old trials:  15


In [ ]:
#df_old = df_old[['number','value','params_dropout','params_learning_rate','params_num_layer','params_optimizer','state','train_loss', 'val_loss', 'Spearman_train','Spearman_val','Pearson_train','Pearson_val']]

In [ ]:
#df_old = df_old.rename(columns={'val_loss': 'valid_loss', 'Spearman_train': 'spearman_train', 'Spearman_val': 'spearman_val', 'Pearson_train':'pearson_train','Pearson_val':'pearson_val'})


In [ ]:
# Seleccionar los valores de df_optuna que no están ya en df_old
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

21


In [ ]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [ ]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

In [ ]:
if len(df_optuna) == len(df_metrics):
  df_complete_updated = pd.concat([df_optuna, df_metrics], axis=1, sort=False)

In [ ]:
df_complete_updated

In [ ]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                 22
value                                                            0.218492
params_dropout                                                   0.325337
params_learning_rate                                             0.000026
params_num_layer                                                        0
params_optimizer                                                  RMSprop
state                                                            COMPLETE
train_loss              [0.6062887161970139, 0.17827804059647828, 0.08...
valid_loss              [0.2674328852444887, 0.35335296764969826, 0.23...
spearman_train          [0.6504019766387401, 0.868394690266739, 0.9349...
spearman_val            [0.8245921715010127, 0.8264217401228285, 0.835...
pearson_train           [0.684871116208399, 0.911718797236082, 0.95635...
pearson_val             [0.8746574743293634, 0.8797325323184438, 0.883...
Name: 22, dtype: object


In [ ]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
#os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
#os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>